<a href="https://colab.research.google.com/github/rhernandez00/Comparative-study-of-dog-vision/blob/main/HemodynamicResponseFunction/HRF_to_visual_stimuli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hemodynamic response to visual stimuli in humans and dogs

This notebook will show you the hemodynamic response to a visual stimuli in the coordinate you select. To use it, go to the top, select Runtime and then Run all (or run every block independently and in order by clicking the "play" button on the left of each of them). The whole process should take about 5 minutes and it runs free on Google's servers.

## Imports

In [8]:
#@title
pip install nilearn

SyntaxError: ignored

In [10]:
pip install --upgrade matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# Some imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import nibabel as nib
import ipywidgets as widgets
from ipywidgets import HBox,VBox
from nilearn import image
from copy import deepcopy
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
from nilearn.plotting import plot_anat, show
import warnings
warnings.filterwarnings('ignore')


print('Downloading files from server')
# downloading the results
!gdown '1FvmZMDOYAMQuPUKzwtP01CVUQJED3h8s&confirm=t' &> /dev/null #dog average data
!gdown '1jtiCkOIX2visobXKhm8FXrDjiqBud2Np&confirm=t' &> /dev/null #dog error data
!gdown '1GA4YwMryE1tGOc8adNCbbXXf3oqFCkKH&confirm=t' &> /dev/null #human average data
!gdown '1G_WIaW_hJHfKu47_a_oR2L_ETBKwhd14&confirm=t' &> /dev/null #human error data

!gdown '1Gk3OPS78RZLt5ACGnkKHpxaauUmtJi25' &> /dev/null #MNI nifti file to use as base
!gdown '1Gj6lMMoLNWY3kUqINg4r3AlX8K703O9N' &> /dev/null #Dog atlas to use as base

print('Loading files into RAM...')

filesPath = '/content'
speciesPossible = ['D','H']
imgTypes = ['err','mean']

niiDict = dict() #this dictionary will contain all 4D images
for specie in speciesPossible:
    niiDict[specie] = dict()
    for imgType in imgTypes:
        fileName = filesPath + '/' + specie + '_HRF_4D' + imgType + '.nii.gz'
        niiDict[specie][imgType] = np.array(nib.load(fileName).dataobj)
        print(fileName + ' loaded')

Loading files to RAM...


## Some functions and code for the script to work

In [7]:
#list of colors. These colors are the same across the experiment in every figure.
colors = [(0.5,0.5,0.5), #grey
          (120.0/255.0,158.0/255.0,2.0/255.0), 
          (72.0/255.0,144.0/255.0,223.0/255.0), 
          (252.0/255.0,42.0/255.0,180.0/255.0),  
          (0.0,0.0,0.0)]

def plotHRF(xVoxD,yVoxD,zVoxD,xVoxH,yVoxH,zVoxH,nColor=2,nColor2=3):
  #Plots the HRF in the voxels xVox, yVox, zVox for each specie
  #creating timeLine
  
  #setting up colors
  colorD = colors[nColor]
  colorH = colors[nColor2]

  voxDict = {'D': {'x':xVoxD, 'y':yVoxD, 'z':zVoxD}, 'H': {'x':xVoxH, 'y':yVoxH, 'z':zVoxH}}
  valDict = dict()
  for specie in speciesPossible:
    valDict[specie] = dict()
    for imgType in imgTypes:
      valDict[specie][imgType] =  niiDict[specie][imgType][voxDict[specie]['x'],voxDict[specie]['y'],voxDict[specie]['z'],:]
  
  fig, ax = plt.subplots(1, 1, figsize=(7.5, 5.4)); #creating axis for fig
  
  #plotting  HRF
  lineD = ax.plot(timeLine,valDict['D']['mean'],color=colorD, label='Dog');
  lineH = ax.plot(timeLine,valDict['H']['mean'],color=colorH, label='Human')

  ax.fill_between(timeLine,valDict['D']['mean']+valDict['D']['err'],valDict['D']['mean']-valDict['D']['err'], alpha=0.4, color=colorD)
  ax.fill_between(timeLine,valDict['H']['mean']+valDict['H']['err'],valDict['H']['mean']-valDict['H']['err'], alpha=0.4, color=colorH)
  plt.ylim([minY,maxY])
  
  #plotting stimuli time
  ax.fill_between(timeLine,np.min(timeLine),np.max(timeLine),
                  where=(timeLine > 0) & (timeLine < minDuration),
                  alpha=0.3, color=[0,0,0], label='All videos are visible')
  ax.fill_between(timeLine,np.min(timeLine),np.max(timeLine),
                  where=(timeLine > minDuration) & (timeLine < maxDuration),
                  alpha=0.15, color=[0,0,0], label='Some videos are visible')
  
  plt.legend()
  
  ax.set_xlabel('Time (s)');
  ax.set_ylabel('Average z-score');
  
  plt.show()

def plotBrain(xD,yD,zD,xH,yH,zH):
  #updates coordinates to use and plots the slices
  positionD = coordinatesToVoxel([xD,yD,zD], ref= atlasFileD)
  positionH = coordinatesToVoxel([xH,yH,zH], ref= atlasFileH)

  display = plot_anat(imgD, display_mode='ortho', cut_coords=[xD,yD,zD], draw_cross=True,vmin=vminD, vmax=vmaxD, title='Dog');
  display = plot_anat(imgH, display_mode='ortho', cut_coords=[xH,yH,zH], draw_cross=True,vmin=vminH, vmax=vmaxH, title='Human');
  plotHRF(positionD[0],positionD[1],positionD[2],positionH[0],positionH[1],positionH[2]);

def coordinatesToVoxel(coords,ref='Barney2mm'):
  #print('Reference used: ' + ref)
  baseX,baseY,baseZ,resolution = getBaseVals(ref)
  coordsVox = []
  if ref == 'Barney2mm': #dogs
    coordsVox.append((coords[0]/resolution) + baseX)
    coordsVox.append((coords[1]/resolution) + baseY)
    coordsVox.append((coords[2]/resolution) + baseZ)
  elif ref == 'MNI2mm': #humans
    coordsVox.append((-1*(coords[0]/resolution) + baseX))
    #coordsVox.append(((coords[0]/resolution) + baseX))
    coordsVox.append((coords[1]/resolution) + baseY)
    coordsVox.append((coords[2]/resolution) + baseZ)
  for i,val in enumerate(coordsVox): coordsVox[i] = int(val) 
  return coordsVox

def getBaseVals(ref):
  if ref == 'Barney2mm': #dogs
    baseX,baseY,baseZ = 15.5, 29.5, 12.0
    resolution = 2
  elif ref == 'MNI2mm': #humans
    baseX,baseY,baseZ = 45.0, 63.0, 36.0
    resolution = 2
  return baseX,baseY,baseZ,resolution

minY,maxY = -0.14,0.175 #Minimum and maximum for Y axis
minDuration,maxDuration = 3.52,5.52 #durations
startTime,endTime,timeStep = -1.0,12.0,0.02  #variables for the plot
timeLine = np.arange(startTime,endTime+timeStep,timeStep)

imagePathD = '/content/BarneyBrain2mm.nii.gz'
atlasFileD = 'Barney2mm'
minXD,maxXD,minYD,maxYD,minZD,maxZD = -27,28,-59,41,-18,31
iniXD,iniYD,iniZD = 0,-29,20
vminD,vmaxD=29.0,1683.0

imagePathH = '/content/MNI152_T1_2mm_brain.nii.gz'
atlasFileH = 'MNI2mm'
minXH,maxXH,minYH,maxYH,minZH,maxZH = -83,83,-110,82,-50,80
iniXH,iniYH,iniZH = 44,-74,-2
vminH,vmaxH=0.0,8339.0

#Loading images to be used as background
imgD = image.load_img(imagePathD) 
imgH = image.load_img(imagePathH)

# Select a coordinate to plot the hemodynamic response

The figure shows the location of the x,y,z coordinates in the dog and human brain. The plot below shows the brain response in those coordinates. The responses are aligned to the onset of the video.

In [8]:
#@title
col1 = VBox([widgets.IntSlider(min=minXD, max=maxXD, step=1, value=iniXD, description='X in dog'),
                    widgets.IntSlider(min=minYD, max=maxYD, step=1, value=iniYD, description='Y in dog'),
                    widgets.IntSlider(min=minZD, max=maxZD, step=1, value=iniZD, description='Z in dog')])

col2 = VBox([widgets.IntSlider(min=minXH, max=maxXH, step=1, value=iniXH, description='X in human'),
                    widgets.IntSlider(min=minYH, max=maxYH, step=1, value=iniYH, description='Y in human'),
                    widgets.IntSlider(min=minZH, max=maxZH, step=1, value=iniZH, description='Z in human')])

out = widgets.interactive_output(plotBrain, {'xD':col1.children[0], 'yD':col1.children[1], 'zD':col1.children[2], 
                                'xH':col2.children[0], 'yH':col2.children[1], 'zH':col2.children[2]})

display(HBox([col1, col2]), out)

Output()